In [2]:
from openai import OpenAI
from langchain.utilities.duckduckgo_search import DuckDuckGoSearchAPIWrapper

import os
from dotenv import load_dotenv
load_dotenv()

OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]

In [32]:
functions = [
    {
        "type": "function",
        "function": {
            "name": "get_ticker",
            "description": "Given the name of a company returns its ticker symbol",
            "parameters": {
                "type": "object",
                "properties": {
                    "company_name": {
                        "type": "string",
                        "description": "The name of the company",
                    }
                },
                "required": ["company_name"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_income_statement",
            "description": "Given a ticker symbol (i.e AAPL) returns the company's income statement.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
            },
        },
    },
{
        "type": "function",
        "function": {
            "name": "get_balance_sheet",
            "description": "Given a ticker symbol (i.e AAPL) returns the company's balance sheet.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_daily_stock_performance",
            "description": "Given a ticker symbol (i.e AAPL) returns the performance of the stock for the last 100 days.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
            },
        },
    },
]


client = OpenAI(api_key=OPENAI_API_KEY)

# assistant = client.beta.assistants.create(
#     name="Investor Assistant",
#     description="당신은 사용자들이 공개적으로 거래되는 회사에 대해 연구하도록 돕고, 그들이 주식을 매수해야 할지 말지 결정하도록 도와줍니다.",
#     model="gpt-3.5-turbo",
#     tools=functions
# )


In [44]:
import yfinance as yfin
import json

def get_ticker(inputs):
    ddg = DuckDuckGoSearchAPIWrapper()
    company_name = inputs["company_name"]
    return ddg.run(f"Ticker symbol of {company_name}")

def get_income_statement(inputs):
    ticker = inputs["ticker"]
    stock = yfin.Ticker(ticker)
    return json.dumps(stock.income_stmt.to_json())

def get_balance_sheet(inputs):
    ticker = inputs["ticker"]
    stock = yfin.Ticker(ticker)
    return json.dumps(stock.balance_sheet.to_json())

def get_daily_stock_performance(inputs):
    ticker = inputs["ticker"]
    stock = yfin.Ticker(ticker)
    return json.dumps(stock.history(period="3mo").to_json())


functions_map = {
    "get_ticker": get_ticker,
    "get_income_statement": get_income_statement,
    "get_balance_sheet": get_balance_sheet,
    "get_daily_stock_performance": get_daily_stock_performance,
}
    

In [34]:
assistant_id = "asst_iTzvqjn9oMAkI6BWHtptBSLt"

thread = client.beta.threads.create(
    messages=[{
        "role":"user",
        "content":"Salesforce 주식이 매수하기 좋은 주식인지 알고 싶습니다"
    }]
)
thread

Thread(id='thread_xuCWibCuTpHT6srFM5u0qhVH', created_at=1721135527, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))

In [36]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant_id
)
run

Run(id='run_PmwXARKZmFq1i2tWpRtXhWAN', assistant_id='asst_iTzvqjn9oMAkI6BWHtptBSLt', cancelled_at=None, completed_at=None, created_at=1721135571, expires_at=1721136171, failed_at=None, incomplete_details=None, instructions=None, last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-3.5-turbo', object='thread.run', parallel_tool_calls=True, required_action=None, response_format='auto', started_at=None, status='queued', thread_id='thread_xuCWibCuTpHT6srFM5u0qhVH', tool_choice='auto', tools=[FunctionTool(function=FunctionDefinition(name='get_ticker', description='Given the name of a company returns its ticker symbol', parameters={'type': 'object', 'properties': {'company_name': {'type': 'string', 'description': 'The name of the company'}}, 'required': ['company_name']}), type='function'), FunctionTool(function=FunctionDefinition(name='get_income_statement', description="Given a ticker symbol (i.e AAPL) returns the company's income statement.", parame

In [41]:
def get_run(run_id, thread_id):
    return client.beta.threads.runs.retrieve(
        run_id=run_id,
        thread_id=thread_id
    )
def get_messages(thread_id):
    messages = client.beta.threads.messages.list(
        thread_id=thread_id,
    )
    messages = list(messages)
    messages.reverse()
    for message in messages:
        print(f"{message.role}:: {message.content[0].text.value}")

def send_message(thread_id, content):
    return client.beta.threads.messages.create(
        thread_id=thread_id,
        role="user",
        content=content
    )

def get_tool_outputs(run_id, thread_id):
    run = get_run(run_id, thread.id)
    outputs = []
    for action in run.required_action.submit_tool_outputs.tool_calls:
        action_id = action.id
        function = action.function
        print(f"Calling function: {function.name} with arg {function.arguments}")
        outputs.append(
            {
                "output": functions_map[function.name](json.loads(function.arguments)),
                "tool_call_id": action_id,
            }
        )
    return outputs


def submit_tool_outputs(run_id, thread_id):
    outpus = get_tool_outputs(run_id, thread_id)
    return client.beta.threads.runs.submit_tool_outputs(
        run_id=run_id, thread_id=thread_id, tool_outputs=outpus
    )
        

In [38]:
get_run(run.id, thread.id).status

'requires_action'

In [43]:
get_messages(thread.id)

user:: Salesforce 주식이 매수하기 좋은 주식인지 알고 싶습니다


In [42]:
send_message(thread.id,"그렇게 해줘")

BadRequestError: Error code: 400 - {'error': {'message': "Can't add messages to thread_xuCWibCuTpHT6srFM5u0qhVH while a run run_PmwXARKZmFq1i2tWpRtXhWAN is active.", 'type': 'invalid_request_error', 'param': None, 'code': None}}